<a href="https://colab.research.google.com/github/fedraameneiro/prueba/blob/main/Proyecto_final_Ciencia_de_Datos_FedraAmeneiro_api_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#API Proyecto diabetes

Vamos a generar una API de interacción con el usuario.
El usuario podrá introducir los datos del paciente y llamar al modelo de predicción con machine learning. El modelo nos dirá si el paciente es o no diabético y lo mostrará por pantalla.



In [1]:
#librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import plotly as pl
import plotly.express as px


In [2]:
#obtener el archivo csv de drive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/file/d/18ymyIaVZ8FK7WhyGltLrJbfx2nyvzjtK/view?usp=drive_link'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('diabetes.csv')

In [3]:
df=pd.read_csv('diabetes.csv', sep=';')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80,0,1,never,"25,19","6,6",140,0
1,Female,54,0,0,No Info,"27,32","6,6",80,0
2,Male,28,0,0,never,"27,32","5,7",158,0
3,Female,36,0,0,current,"23,45",5,155,0
4,Male,76,1,1,current,"20,14","4,8",155,0


Tenemos 4 variables categoricas y la variable objetivo. Y 3 variables numericas.

Antes de nada voy a chequear aquellas  variables numericas que aparecen como objetos y los voy a modificar. bmi y HbA1c_level deberían ser enteros. Les voy a cambiar la coma por un punto.

In [4]:
df.bmi=df.bmi.str.replace(',','.')
df.HbA1c_level=df.HbA1c_level.str.replace(',','.')
df.bmi=df["bmi"].astype(float)
df.HbA1c_level=df["HbA1c_level"].astype(float)
df.dtypes

gender                  object
age                      int64
hypertension             int64
heart_disease            int64
smoking_history         object
bmi                    float64
HbA1c_level            float64
blood_glucose_level      int64
diabetes                 int64
dtype: object

In [5]:
df_diabetes=df[df.diabetes==1]
df_nodiabetes=df[df.diabetes==0]
df_diabetes.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
6,Female,44,0,0,never,19.31,6.5,200,1
26,Male,67,0,1,not current,27.32,6.5,200,1
38,Male,50,1,0,current,27.32,5.7,260,1
40,Male,73,0,0,former,25.91,9.0,160,1
53,Female,53,0,0,former,27.32,7.0,159,1


Eliminamos aquellas muestras con genero other.

In [6]:
df['gender'].unique()

array(['Female', 'Male', 'Other'], dtype=object)

In [7]:
df=df[df.gender!='Other']

Vemos la correlación entre variables numéricas y confirmamos que las mejores variables que se relacionan con la diabetes son el nivel HbA1c_level y el nivel de glucosa en sangre.

In [8]:
# Importa las bibliotecas necesarias
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [9]:
X=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']

# Normaliza las características (es importante para las redes neuronales)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
# Construir el modelo de red neuronal
modelo = Sequential()
modelo.add(Dense(32, input_dim=4, activation='relu'))
modelo.add(Dense(16, activation='relu'))
modelo.add(Dense(1, activation='sigmoid'))



In [11]:
# Compila el modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Entrena el modelo con los datos de entrenamiento. epoch=1000
modelo.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evalúa el modelo con los datos de prueba
precisionRedesNeuronales = modelo.evaluate(X_test, y_test) [1]
print(f'Precisión del modelo: {precisionRedesNeuronales}')


Epoch 1/100
2000/2000 [==============================] - 16s 7ms/step - loss: 0.1314 - accuracy: 0.9564 - val_loss: 0.1053 - val_accuracy: 0.9657
Epoch 2/100
2000/2000 [==============================] - 9s 5ms/step - loss: 0.0990 - accuracy: 0.9667 - val_loss: 0.0941 - val_accuracy: 0.9691
Epoch 3/100
2000/2000 [==============================] - 7s 3ms/step - loss: 0.0913 - accuracy: 0.9695 - val_loss: 0.0895 - val_accuracy: 0.9714
Epoch 4/100
2000/2000 [==============================] - 5s 3ms/step - loss: 0.0876 - accuracy: 0.9708 - val_loss: 0.0870 - val_accuracy: 0.9717
Epoch 5/100
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0861 - accuracy: 0.9714 - val_loss: 0.0858 - val_accuracy: 0.9721
Epoch 6/100
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0850 - accuracy: 0.9716 - val_loss: 0.0853 - val_accuracy: 0.9721
Epoch 7/100
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0846 - accuracy: 0.9717 - val_loss: 0.0852 - val_a

Guardamos en un archivo requirements.txt las versiones de las librerías utilizadas, para usar los mismos en la ejecución de mi API y no me de errores.

In [19]:
!python --version

Python 3.10.12


In [23]:
!pip list


Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi

In [24]:
!pip freeze > requirements.txt

Almacenamos mi modelo en un archivo 'modelo.h5'. Intenté usar la libreria joblib y almacenarlo en modelo.joblib pero me dio problemas al utilizarlo.

In [14]:
import joblib
joblib.dump(modelo, 'modelo.pkl')

['modelo.pkl']

In [15]:
import pickle
pickle.dump(modelo,open('modelo.sav','wb'))

In [22]:
from keras.models import load_model

# Guardar el modelo
modelo.save('modelo.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Este archivo lo almacenaremos en el servidor que vamos a usar para ejecutar nuestra api.